In [10]:
import pandas as pd

In [2]:
#load all ti matches
matches_df = pd.read_json('all_matches.json')
matches_df.head()



,match_id,barracks_status_dire,barracks_status_radiant,chat,cluster,cosmetics,dire_score,dire_team_id,draft_timings,duration,...,players,patch,region,all_word_counts,my_word_counts,throw,loss,replay_url,comeback,stomp
0,6806403033,15,63,"[{'time': -968, 'type': 'chatwheel', 'key': '5...",154,"{'647': 129, '6496': 128, '6498': 128, '6526':...",27,8254400,"[{'order': 1, 'pick': False, 'active_team': 3,...",2053,...,"[{'match_id': 6806403033, 'player_slot': 0, 'a...",51,5,"{'glhf': 1, 'glgl': 1, 'gg': 5, 'wp': 1, 'ggwp...",{},10360.0,4933.0,http://replay154.valve.net/570/6806403033_1122...,NaN,NaN
1,6806374409,51,11,"[{'time': -759, 'type': 'chatwheel', 'key': '5...",154,"{'647': 131, '746': 130, '6159': 0, '6162': 0,...",39,7390454,"[{'order': 1, 'pick': False, 'active_team': 3,...",3732,...,"[{'match_id': 6806374409, 'player_slot': 0, 'a...",51,5,"{'glhf': 1, 'gg': 5}",{},9381.0,6601.0,http://replay154.valve.net/570/6806374409_1647...,NaN,NaN
2,6806377510,63,0,"[{'time': -139, 'type': 'chatwheel', 'key': '5...",154,"{'647': 128, '5105': 4, '5412': 3, '6669': 4, ...",43,6209804,"[{'order': 1, 'pick': False, 'active_team': 3,...",2879,...,"[{'match_id': 6806377510, 'player_slot': 0, 'a...",51,5,"{'glhf': 1, 'gl': 1, 'hf': 1, 'glgl': 1, 'soun...",{},NaN,NaN,http://replay154.valve.net/570/6806377510_1350...,7288.0,20699.0
3,6806364123,63,0,"[{'time': -86, 'type': 'chatwheel', 'key': '54...",154,"{'600': 3, '647': 130, '4007': 2, '5927': 131,...",31,7119388,"[{'order': 1, 'pick': False, 'active_team': 3,...",2285,...,"[{'match_id': 6806364123, 'player_slot': 0, 'a...",51,5,"{'glhf': 2, 'gl': 1, 'gg': 7}",{},NaN,NaN,http://replay154.valve.net/570/6806364123_1777...,458.0,26622.0
4,6806365638,48,63,"[{'time': -87, 'type': 'chatwheel', 'key': '85...",154,"{'647': 0, '4990': 131, '4992': 131, '6833': 1...",23,8599101,"[{'order': 1, 'pick': False, 'active_team': 3,...",2390,...,"[{'match_id': 6806365638, 'player_slot': 0, 'a...",51,5,"{'hfhfhf': 1, 'gl': 1, 'hfffff': 1, 'glhf': 2,...",{},1376.0,15328.0,http://replay154.valve.net/570/6806365638_8319...,NaN,NaN


In [3]:
def get_winner(matchid):
    match_data = matches_df[matches_df['match_id']==matchid]
    radiant_win = match_data['radiant_win'].to_string(index=False)
    winner = 1
    if radiant_win == 'True':
        winner = 0
    return winner



In [4]:
#define heroes and teams
heroes_df = pd.read_json('heroes.json')
heroes_df = heroes_df[['id','localized_name']]
heroes_df.rename(columns={"localized_name":"hero_name"},inplace=True)
heroes_df.head()


,id,hero_name
0,1,Anti-Mage
1,2,Axe
2,3,Bane
3,4,Bloodseeker
4,5,Crystal Maiden


In [5]:
#create pickbans df
picks_bans = matches_df.set_index('match_id').to_dict()['picks_bans']
picks_bans
picks = []
bans = []
for i in picks_bans:
    winner = get_winner(i)
    for j in picks_bans[i]:
        if j['is_pick']:
            pick = {}
            pick['match_id'] = i
            pick['order'] = j['order']
            pick['hero_id'] = j['hero_id']
            pick['team'] = j['team']
            if winner == j['team']:
                pick['win'] = 1
            else:
                pick['win'] = 0
            picks.append(pick)
        elif not j['is_pick']:
            ban = {}
            ban['match_id'] = i
            ban['order'] = j['order']
            ban['hero_id'] = j['hero_id']
            ban['team'] = j['team']
            bans.append(ban)
                      




In [6]:
picks_df = pd.DataFrame(picks)
heroes_df['pick_count'] = heroes_df['id'].map(picks_df['hero_id'].value_counts())
wins = picks_df[['hero_id','win']].groupby(['hero_id']).sum()
wins = wins.to_dict()
heroes_df['wins'] = heroes_df['id'].map(wins['win'])
heroes_df['winrate'] = round(heroes_df['wins'] / heroes_df['pick_count'] * 100,2)

bans_df = pd.DataFrame(bans)
heroes_df['ban_count'] = heroes_df['id'].map(bans_df['hero_id'].value_counts())
heroes_df

,id,hero_name,pick_count,wins,winrate,ban_count
0,1,Anti-Mage,1.0,1.0,100.00,NaN
1,2,Axe,NaN,NaN,NaN,1.0
2,3,Bane,1.0,0.0,0.00,NaN
3,4,Bloodseeker,1.0,0.0,0.00,12.0
4,5,Crystal Maiden,3.0,2.0,66.67,NaN
...,...,...,...,...,...,...
118,128,Snapfire,10.0,5.0,50.00,1.0
119,129,Mars,6.0,2.0,33.33,8.0
120,135,Dawnbreaker,3.0,1.0,33.33,3.0
121,136,Marci,11.0,7.0,63.64,21.0


# The most pick Hero is:

In [7]:
hero_picks = heroes_df[['hero_name','pick_count']]
pick_max = hero_picks['pick_count'].max()
hero_picks[hero_picks['pick_count']==pick_max]


,hero_name,pick_count
122,Primal Beast,13.0


# The most banned hero is:

In [8]:
ban_picks = heroes_df[['hero_name','ban_count']]
ban_max = ban_picks['ban_count'].max()
ban_picks[ban_picks['ban_count']==ban_max]

,hero_name,ban_count
31,Enigma,23.0
63,Batrider,23.0


# The highest winrate hero is:

In [9]:
winrate_df = heroes_df[heroes_df['pick_count']>5].sort_values('winrate',ascending=False)
winrate_df = winrate_df[['hero_name','winrate']]
win_max = winrate_df['winrate'].max()
winrate_df[winrate_df['winrate']==win_max]

,hero_name,winrate
18,Tiny,71.43
